In [1]:
# Importación de librerías
import pandas as pd
import numpy as np

# Cálculo de stock e impacto económico

Cargamos los ficheros de RMSE de modelos ml y naives.

In [2]:
RMSE_naive = pd.read_csv('Predicciones_Test_Naive_Modelo.csv')
RMSE_ML = pd.read_csv('Mejores_Modelos_por_Producto.csv')

In [3]:
#Ordenamos por producto
RMSE_ML.sort_values(by=['producto'], inplace=True)
RMSE_ML

,producto,modelo,RMSE
0,1,RF,7.720392
1,2,RF,5.574722
2,3,DT,4.496971
3,4,RF,3.376366
4,5,DT,3.944046
...,...,...,...
889,996,RF,2.144110
890,997,RF,1.537066
891,998,RF,2.132964
892,999,RF,2.993253


In [4]:
RMSE_naive.sort_values(by=['producto'], inplace=True)
RMSE_naive

,producto,RMSE
0,1,9.570524
1,2,8.206589
2,3,6.502190
3,4,3.691420
4,5,5.332608
...,...,...
889,996,2.393292
890,997,1.891018
891,998,2.631359
892,999,4.141485


Importamos los datos de ciclo de aprovisionamiento

In [5]:
DatosCicloAprovisionamiento = pd.read_excel('DatosCicloAprovisionamiento.xlsx')


# Calculamos la raiz del ciclo de aprovisionamiento
DatosCicloAprovisionamiento['RaizCicloAprovisionamiento'] = np.sqrt(DatosCicloAprovisionamiento['diasEntrePedidos'] * DatosCicloAprovisionamiento['diasLeadtime'])

DatosCicloAprovisionamiento.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento
0,1,14,15,14.491377
1,2,14,15,14.491377
2,3,14,15,14.491377
3,4,14,15,14.491377
4,5,14,15,14.491377


In [6]:
# Ahora calculamos las unidades de stock
factor_servicio = 1.64  # Para un nivel de servicio del 95%

DatosCicloAprovisionamiento_ML = DatosCicloAprovisionamiento.merge(RMSE_ML, on='producto')
DatosCicloAprovisionamiento_ML['UnidadesStock'] = factor_servicio * DatosCicloAprovisionamiento_ML['RMSE'] * DatosCicloAprovisionamiento_ML['RaizCicloAprovisionamiento']
DatosCicloAprovisionamiento_ML.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock
0,1,14,15,14.491377,RF,7.720392,183.481740
1,2,14,15,14.491377,RF,5.574722,132.488045
2,3,14,15,14.491377,DT,4.496971,106.874362
3,4,14,15,14.491377,RF,3.376366,80.242244
4,5,14,15,14.491377,DT,3.944046,93.733645


In [7]:
# Importamos el archivo con los datos de precios
DatosPrecioMedio = pd.read_excel('DatosPrecioMedio.xlsx')
DatosPrecioMedio.head()

,producto,eurPrecioMedio
0,1,68.730000
1,2,148.330000
2,3,169.000000
3,4,0.604383
4,5,4.553314


In [8]:
#Calculamos el coste diario
DatosPrecioMedio_ML = DatosPrecioMedio.merge(DatosCicloAprovisionamiento_ML, on='producto')

factor = 0.05
DatosPrecioMedio_ML['CosteDiarioStock'] = DatosPrecioMedio_ML['UnidadesStock'] * DatosPrecioMedio_ML['eurPrecioMedio'] * factor 

DatosPrecioMedio_ML.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock,CosteDiarioStock
0,1,68.730000,14,15,14.491377,RF,7.720392,183.481740,630.535001
1,2,148.330000,14,15,14.491377,RF,5.574722,132.488045,982.597584
2,3,169.000000,14,15,14.491377,DT,4.496971,106.874362,903.088361
3,4,0.604383,14,15,14.491377,RF,3.376366,80.242244,2.424852
4,5,4.553314,14,15,14.491377,DT,3.944046,93.733645,21.339937


In [9]:
# Calculo del coste anual

DatosPrecioMedio_ML['CosteAnualStock'] = DatosPrecioMedio_ML['CosteDiarioStock'] * 365
DatosPrecioMedio_ML.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,modelo,RMSE,UnidadesStock,CosteDiarioStock,CosteAnualStock
0,1,68.730000,14,15,14.491377,RF,7.720392,183.481740,630.535001,230145.275242
1,2,148.330000,14,15,14.491377,RF,5.574722,132.488045,982.597584,358648.118274
2,3,169.000000,14,15,14.491377,DT,4.496971,106.874362,903.088361,329627.251605
3,4,0.604383,14,15,14.491377,RF,3.376366,80.242244,2.424852,885.071049
4,5,4.553314,14,15,14.491377,DT,3.944046,93.733645,21.339937,7789.077107


In [10]:
# Suma de los costes de una año

sum_coste_anual_ML = DatosPrecioMedio_ML['CosteAnualStock'].sum()
print(f'El coste anual total de stock es: {sum_coste_anual_ML} euros')

El coste anual total de stock es: 23471238.689149693 euros


Hacemos lo mismo con el modelo Naive

In [11]:
# Calculamos las unidades de stock
factor_servicio = 1.64  # Para un nivel de servicio del 95%

DatosCicloAprovisionamiento_NF = DatosCicloAprovisionamiento.merge(RMSE_naive, on='producto')
DatosCicloAprovisionamiento_NF['UnidadesStock'] = factor_servicio * DatosCicloAprovisionamiento_NF['RMSE'] * DatosCicloAprovisionamiento_NF['RaizCicloAprovisionamiento']
DatosCicloAprovisionamiento_NF.head()

,producto,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock
0,1,14,15,14.491377,9.570524,227.451722
1,2,14,15,14.491377,8.206589,195.036625
2,3,14,15,14.491377,6.502190,154.530135
3,4,14,15,14.491377,3.691420,87.729765
4,5,14,15,14.491377,5.332608,126.734006


In [12]:
#Calculamos el coste diario
DatosPrecioMedio_NF = DatosPrecioMedio.merge(DatosCicloAprovisionamiento_NF, on='producto')

factor = 0.05
DatosPrecioMedio_NF['CosteDiarioStock'] = DatosPrecioMedio_NF['UnidadesStock'] * DatosPrecioMedio_NF['eurPrecioMedio'] * factor 

DatosPrecioMedio_NF.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock,CosteDiarioStock
0,1,68.730000,14,15,14.491377,9.570524,227.451722,781.637842
1,2,148.330000,14,15,14.491377,8.206589,195.036625,1446.489131
2,3,169.000000,14,15,14.491377,6.502190,154.530135,1305.779638
3,4,0.604383,14,15,14.491377,3.691420,87.729765,2.651119
4,5,4.553314,14,15,14.491377,5.332608,126.734006,28.852988


In [13]:
# Calculo del coste anual

DatosPrecioMedio_NF['CosteAnualStock'] = DatosPrecioMedio_NF['CosteDiarioStock'] * 365
DatosPrecioMedio_NF.head()

,producto,eurPrecioMedio,diasEntrePedidos,diasLeadtime,RaizCicloAprovisionamiento,RMSE,UnidadesStock,CosteDiarioStock,CosteAnualStock
0,1,68.730000,14,15,14.491377,9.570524,227.451722,781.637842,285297.812379
1,2,148.330000,14,15,14.491377,8.206589,195.036625,1446.489131,527968.532707
2,3,169.000000,14,15,14.491377,6.502190,154.530135,1305.779638,476609.567961
3,4,0.604383,14,15,14.491377,3.691420,87.729765,2.651119,967.658326
4,5,4.553314,14,15,14.491377,5.332608,126.734006,28.852988,10531.340579


In [14]:
# Suma de los costes de una año

sum_coste_anual_NF = DatosPrecioMedio_NF['CosteAnualStock'].sum()
print(f'El coste anual total de stock es: {sum_coste_anual_NF} euros')

El coste anual total de stock es: 33892712.41255322 euros
